# 문제 6

[Kaggle 형] train_prob.csv로 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend # 오류 발생시!!!: !pip install --upgrade mlxtend 
import sys
import xgboost as xgb # 오류 발생시!!!: !pip install --upgrade xgboost 

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
# 데이터를 식별할 수 있는(유일성이 있는) 컬럼은 인덱스로 쓸만합니다.  index_col  = 'id'
df_train = pd.read_csv('train_prob.csv', index_col = 'id')
df_test = pd.read_csv('test_prob.csv', index_col = 'id')
display(df_train.head())
display(df_test.head())

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
id,,,,,,,,,,,,,,,,,,,,,
0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
id,,,,,,,,,,,,,,,,,,,,,
16115,D,76.89,material_7,material_5,6,6,16,2,7,17.427,...,18.241,11.536,14.690,19.622,11.082,16.323,15.633,16.153,15.640,648.974
16116,D,96.13,material_7,material_5,6,6,9,10,5,18.014,...,19.056,10.746,NaN,18.475,10.174,15.710,16.274,12.229,17.204,741.103
16117,D,124.40,material_7,material_5,6,6,10,15,0,18.374,...,20.689,10.240,16.035,18.224,11.005,15.925,16.257,16.338,17.653,NaN
16118,D,175.84,material_7,material_5,6,6,4,8,15,18.056,...,19.442,12.015,18.195,19.410,10.761,16.176,16.926,13.813,13.615,NaN
16119,D,118.70,material_7,material_5,6,6,15,8,5,16.513,...,19.007,12.615,15.775,18.778,9.683,NaN,17.066,14.690,18.798,NaN


In [3]:
# 문제 1의 선택 과정으로 도출된 유의미한 결측값
df_train = df_train.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna(),
)
df_test = df_test.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna(),
)

In [4]:
df_train['product_code'].value_counts()

C    5765
E    5343
B    5250
A    5100
Name: product_code, dtype: int64

In [5]:
df_test['product_code'].value_counts()

D    5112
Name: product_code, dtype: int64

In [6]:
# 방법 2: groupby ~ apply ~ fit_transform
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
# product_code에 따른 measurement3-9, measurement17의 선형 관계를 이용

# basetable의 measurement_3~9와 measurement_17 결측치를 복원
# 입력: measurement_3 ~ 9, measurement_17
X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
imp = IterativeImputer(
    estimator = LinearRegression(fit_intercept = True), 
    random_state=123
)
# apply에서 imp가 transform 해서 준 타입이 2차원 numpy array입니다.
# 이는 groupby ~ apply 에서 처리를 못해줍니다.
# 2차원 numpy array를 DataFrame으로 만들어주면
# apply에서 DataFrame으로 재구성하여 넘겨줍니다.
# 모델을 적용하여 결측이 처리된 데이터프레임을 만듭니다.
df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index = x.index, columns = x.columns)
)
df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index = x.index, columns = x.columns)
)

In [7]:
# 방법1: groupby ~ transform
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[X_mean] = df_train.groupby('product_code')[X_mean].transform(
    lambda x: x.fillna(x.mean())
)
df_test[X_mean] = df_test.groupby('product_code')[X_mean].transform(
    lambda x: x.fillna(x.mean())
)

In [18]:
m = pd.concat([df_train['loading'], df_test['loading']]).mean()
df_train['loading'] = df_train['loading'].fillna(m)
df_test['loading'] = df_test['loading'].fillna(m)

In [20]:
# np.log(loading)
df_train['loading_log'] = np.log(df_train['loading'])
df_test['loading_log'] = np.log(df_test['loading'])

In [21]:
df_train.isna().sum()

product_code      0
loading           0
attribute_0       0
attribute_1       0
attribute_2       0
attribute_3       0
measurement_0     0
measurement_1     0
measurement_2     0
measurement_3     0
measurement_4     0
measurement_5     0
measurement_6     0
measurement_7     0
measurement_8     0
measurement_9     0
measurement_10    0
measurement_11    0
measurement_12    0
measurement_13    0
measurement_14    0
measurement_15    0
measurement_16    0
measurement_17    0
failure           0
na_1              0
na_2              0
loading_log       0
dtype: int64

문제2:  버림: attribute_0, attribute_1

문제3: LR STD\['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17'\] PT\['na_1'\]

문제4: LDA 분류, 변환 / PCA: LR + loading + PCA n_components = 7 STD \[measurement_0 ~ 17\]

문제5: RF  {'n_estimators': 15, 'max_depth': 7, 'min_samples_split': 512} loading + \[measurement_0 ~ 17\] + na_1, na_2

In [13]:
X_all = df_test.columns.tolist()

# Group KFold

In [14]:
from sklearn.model_selection import GroupKFold

gkf = GroupKFold(4)
for train_idx, valid_idx in gkf.split(df_train[X_all], df_train['failure'], groups = df_train['product_code']):
    print(
        df_train.iloc[train_idx]['product_code'].unique(), df_train.iloc[valid_idx]['product_code'].unique()
    )

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


In [24]:
# LR STD['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17'] PT['na_1']
from sklearn.model_selection import cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
    ('pt', 'passthrough', ['na_1'])
])

clf_lr = make_pipeline(
    ct, LogisticRegression(solver ='lbfgs')
)
cross_validate(
    clf_lr, df_train[X_all], df_train['failure'], groups = df_train['product_code'], scoring = 'roc_auc',
    cv = gkf, return_train_score=True
)

{'fit_time': array([0.02512193, 0.0399797 , 0.02699065, 0.01993132]),
 'score_time': array([0.00970268, 0.        , 0.00977874, 0.        ]),
 'test_score': array([0.58821746, 0.58491734, 0.58894014, 0.59540058]),
 'train_score': array([0.59262252, 0.59350804, 0.59192438, 0.58956303])}